In [14]:
# The basics
import numpy as np
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Softmax, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Bidirectional, LSTM, Dropout

# utils
import os

In [4]:
isear = pd.read_csv('../data/raw/isear.csv', sep='|', error_bad_lines=False, usecols=['Field1', 'SIT', 'EMOT'])

In [5]:
number_of_classes = len(isear.EMOT.unique())

In [6]:
maxlen = 1000
max_words = 10000

In [7]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(isear['SIT'])
sequences = tokenizer.texts_to_sequences(isear['SIT'])

In [8]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9063 unique tokens.


In [9]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post')

In [10]:
x_train, x_test, y_train, y_test = train_test_split(data, isear['EMOT'])

In [11]:
glove_dir = '../data/external'
embeddings_index = {}

f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [12]:
embeddings_index['king'] - embeddings_index['man'] + embeddings_index['woman'] - embeddings_index['queen']

array([ 0.03882596, -0.91902995,  0.25977004,  0.04227898,  0.13896999,
        0.20638007,  0.02683   , -0.03402001, -0.09974198,  0.08821005,
        0.007544  , -0.36111003,  0.320001  , -0.6568099 ,  0.63689005,
        0.33902   , -0.7714    , -0.59767103,  0.711054  ,  0.44857004,
       -0.08957994, -0.01644999, -0.08847399, -0.17911002,  0.43223003,
       -0.28219986, -0.09410012, -0.9570599 ,  0.01775998,  0.32842597,
        0.11489999, -0.36365002, -0.52906007,  0.11805284, -0.10499001,
       -0.219787  , -0.33020002, -0.137476  ,  0.10705006,  0.92508006,
       -0.223916  ,  0.71121997, -0.29156998,  0.24670503, -0.05374995,
        0.2274299 , -0.09986001,  0.33875012, -0.69724536,  0.10499996],
      dtype=float32)

In [13]:
embedding_dim = 50 # if chaning this, update the file name above 

embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Model creation time

In [16]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.2))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(number_of_classes + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 50)          500000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 256)         183296    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 994, 32)           57376     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 198, 32)           0         
_________________________________________________________________
dropout (Dropout)            (None, 198, 32)           0         
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [17]:
model.fit(x_train, to_categorical(y_train),
          epochs=10,
          batch_size=32,
          validation_data=(x_test, to_categorical(y_test)))

Train on 5749 samples, validate on 1917 samples
Epoch 1/10
5749/5749 [==============================] - 225s 39ms/sample - loss: 1.9416 - accuracy: 0.1891 - val_loss: 1.8031 - val_accuracy: 0.2801
Epoch 2/10
5749/5749 [==============================] - 223s 39ms/sample - loss: 1.5069 - accuracy: 0.4046 - val_loss: 1.4436 - val_accuracy: 0.4559
Epoch 3/10
5749/5749 [==============================] - 226s 39ms/sample - loss: 1.1046 - accuracy: 0.5893 - val_loss: 1.2814 - val_accuracy: 0.5389
Epoch 4/10
5749/5749 [==============================] - 221s 38ms/sample - loss: 0.8094 - accuracy: 0.7193 - val_loss: 1.2845 - val_accuracy: 0.5446
Epoch 5/10
5749/5749 [==============================] - 217s 38ms/sample - loss: 0.6117 - accuracy: 0.7965 - val_loss: 1.3434 - val_accuracy: 0.5493
Epoch 6/10
5749/5749 [==============================] - 221s 39ms/sample - loss: 0.4702 - accuracy: 0.8478 - val_loss: 1.4620 - val_accuracy: 0.5357
Epoch 7/10
5749/5749 [==============================] - 22

In [18]:
y_pred = model.predict_classes(x_test)
y_pred

array([2, 7, 6, ..., 6, 5, 3])

In [19]:
confusion_matrix(y_test, y_pred)

array([[182,   6,   4,  32,  12,  33,  16],
       [  8, 153,   8,  16,  22,  30,  27],
       [ 14,  13,  90,  16,  47,  47,  52],
       [ 22,  11,  13, 139,  21,  32,  27],
       [ 14,  11,  23,  32, 158,  30,  16],
       [ 19,  11,  30,   8,  27, 138,  44],
       [ 12,  10,  15,  12,  14,  57, 143]])

In [21]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.67      0.64      0.65       285
           2       0.71      0.58      0.64       264
           3       0.49      0.32      0.39       279
           4       0.55      0.52      0.53       265
           5       0.52      0.56      0.54       284
           6       0.38      0.50      0.43       277
           7       0.44      0.54      0.49       263

    accuracy                           0.52      1917
   macro avg       0.54      0.52      0.52      1917
weighted avg       0.54      0.52      0.52      1917

